# Data curation: MDS part III scores

Form 238v1: MDS-UPDRS Scoring Summary
Form 508v1: MDS-UPDRS Part 3 Scoring Summary

In [100]:
# import packages
import os
import platform # don't need
import pandas as pd
import re
import h5py #save hdf5 but pandas has a function...
import numpy as np

In [101]:
filename = r'//FS2.smpp.local\RTO\CIS-PD MUSC\codelist.sas7bdat'
codelist = pd.read_sas(filename)
codelist.columns = codelist.columns.str.replace('z', '')
codelist = codelist.applymap(lambda x: x.decode() if isinstance(x, bytes) else x)
filename = r'//FS2.smpp.local\RTO\CIS-PD MUSC\datadictionary.sas7bdat'
datadict = pd.read_sas(filename, format = 'sas7bdat', encoding='iso-8859-1')
datadict.columns = datadict.columns.str.replace('z', '')

In [102]:
# load decoded forms
filename = r'//FS2.smpp.local\RTO\CIS-PD MUSC\decoded_forms\form238.h5'
form238 = pd.read_hdf(filename)
filename = r'//FS2.smpp.local\RTO\CIS-PD MUSC\decoded_forms\form508.h5'
form508 = pd.read_hdf(filename)
form238 = form238.rename(index=str, columns={'SubjectCode':'Subject ID',
                                  'VisitNm':'Visit'})
form508 = form508.rename(index=str, columns={'SubjectCode':'Subject ID',
                                  'VisitNm':'Visit'})

In [ ]:
codelist.head()

In [ ]:
datadict.head()

In [ ]:
datadict.loc[datadict.FormID == 238.0]

# Check form 238 and 508 contents

In [ ]:
form238.head(2)

In [ ]:
form508.head(2)

# Modify forms 508 and 238

# Keep Q29-32 on 238
Form 508
- Qa Data collected - N, Y
- Qb Date of assessment
- Q01 state
- Q02 hours
- Qc comments
Form 238
Q29 on medication
Q30 critical state
Q31 on Levadopa
Q32 min

In [ ]:
## Do not drop - 

# drop 508 columns
form508 = form508.drop(columns=['Q02', 'Q01', 'DataCollected', 'Qb2_UTC', 'FormTime'])

In [ ]:
# drop form238 columns

# Did not drop Q29-32, so NaN values will be on Form 508 data

# delete Q32 which is 3.C1 Minutes since last levodopa dose
form238 = form238.drop(columns=['Q02', 'Q03', 'Q04', 'Q05',
'Q06', 'Q07', 'Q09', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16',
'Q17', 'Q18', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26',
'Q27', 'Q28',
'Q69', 'Q70', 'Q71', 'Q72',
'Q73', 'Q74', 'DataCollected', 'Q01', 'Q08'])

In [ ]:
# Part 1-2
# Qa, Qb, Q01 - Q28
# Part 4
# Q69 - Q74, Qc

In [ ]:
# concatenate forms 508 and 238 dataframes
combo = pd.concat([form238, form508])

In [ ]:
oldname = ['Q33', 'Q34', 'Q35', 'Q36', 'Q37', 'Q38', 'Q39', 'Q40', 'Q41', 
           'Q42', 'Q43', 'Q44', 'Q45', 'Q46', 'Q47', 'Q48', 'Q49', 'Q50', 
           'Q51', 'Q52', 'Q53', 'Q54', 'Q55', 'Q56', 'Q57', 'Q58', 'Q59', 
           'Q60', 'Q61', 'Q62', 'Q63', 'Q64', 'Q65', 'Q66', 'Q67', 'Q68']

In [ ]:
# change column names for Q33-68 with 3.13 etc.
newname = [# questions 33-34
'3.1',
'3.2',
# questions 35-39
'3.3 Neck',
'3.3 Right Upper Extremity',
'3.3 Left Upper Extremity',
'3.3 Right Lower Extremity',
'3.3 Left Lower Extremity',
# questions 40-41
'3.4 Right Hand',
'3.4 Left Hand',
# questions 42-43
'3.5 Right Hand',
'3.5 Left Hand',
# questions 44-45
'3.6 Right Hand',
'3.6 Left Hand',
# questions 46-47
'3.7 Right Foot',
'3.7 Left Foot',
# questions 48-49
'3.8 Right Leg',
'3.8 Left Leg',
# questions 50-55
'3.9',
'3.10',
'3.11',
'3.12',
'3.13',
'3.14',
# questions 56-57
'3.15 Right Hand',
'3.15 Left Hand',
# questions 58-59
'3.16 Right Hand',
'3.16 Left Hand',
# questions 60-64
'3.17 Right Upper Extremity',
'3.17 Left Upper Extremity',
'3.17 Right Lower Extremity',
'3.17 Left Lower Extremity',
'3.17 Lip-Jaw',
# question 65
'3.18',
# questions 66-67
'3.19A',
'3.19B',
# question 68
'3.20']

In [ ]:
col_dict = dict(zip(oldname,newname))
print(col_dict)

In [ ]:
combo = combo.rename(index=str, columns=col_dict)

In [ ]:
# get rid of decimals
columns = ['Subject ID', 'SiteID']
for col in columns:
    combo[col] = combo[col].apply(lambda x: x if pd.isnull(x) else int(x))

# keeps columns float but removes decimals
pd.set_option('precision', 0)

In [ ]:
combo.head(2)

In [ ]:
# save file as updrs_part3.h5
filename = r'//FS2.smpp.local\RTO\CIS-PD MUSC\decoded_forms\updrs_part3.h5'
# save pd.read_hdf(filename)
with open(filename,'wb') as f:
    combo.to_hdf(filename, key='combo', mode='w')

In [ ]:
# open file
filename = r'//FS2.smpp.local\RTO\CIS-PD MUSC\decoded_forms\updrs_part3.h5'
pd.read_hdf(filename, 'combo')

# Create updrs_124
- Use Form 238
- Remove part 3
- Keep parts 1, 2, 4

Part 1-2
Qa, Qb, Q01 - Q28
Part 4
Q69 - Q74, Qc

In [261]:
filename = r'//FS2.smpp.local\RTO\CIS-PD MUSC\decoded_forms\form238.h5'
f238 = pd.read_hdf(filename)
f238 = f238.rename(index=str, columns={'SubjectCode':'Subject ID',
                                  'VisitNm':'Visit'})

In [262]:
f238.columns

Index(['Subject ID', 'SiteID', 'Visit', 'FormDate', 'Q02', 'Q03', 'Q04', 'Q05',
       'Q06', 'Q07', 'Q09', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16',
       'Q17', 'Q18', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26',
       'Q27', 'Q28', 'Q32', 'Q33', 'Q34', 'Q35', 'Q36', 'Q37', 'Q38', 'Q39',
       'Q40', 'Q41', 'Q42', 'Q43', 'Q44', 'Q45', 'Q46', 'Q47', 'Q48', 'Q49',
       'Q50', 'Q51', 'Q52', 'Q53', 'Q54', 'Q55', 'Q56', 'Q57', 'Q58', 'Q59',
       'Q60', 'Q61', 'Q62', 'Q63', 'Q64', 'Q65', 'Q69', 'Q70', 'Q71', 'Q72',
       'Q73', 'Q74', 'DataCollected', 'Q01', 'Q08', 'Q29', 'Q30', 'Q31', 'Q66',
       'Q67', 'Q68'],
      dtype='object')

In [ ]:
for i, k in enumerate(f238.columns):
    print(i, k)

In [263]:
# remove columns Q29:31, Q32:Q65, Q66:Q68
f238.drop(f238.columns[30:64], axis=1, inplace=True)
f238 = f238.drop(columns=['Q29','Q30','Q31','Q66', 'Q67', 'Q68'])

In [264]:
f238.columns

Index(['Subject ID', 'SiteID', 'Visit', 'FormDate', 'Q02', 'Q03', 'Q04', 'Q05',
       'Q06', 'Q07', 'Q09', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16',
       'Q17', 'Q18', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26',
       'Q27', 'Q28', 'Q69', 'Q70', 'Q71', 'Q72', 'Q73', 'Q74', 'DataCollected',
       'Q01', 'Q08'],
      dtype='object')

In [265]:
f238 = f238[['Subject ID', 'SiteID', 'Visit', 'FormDate', 'Q01', 'Q02', 'Q03', 'Q04', 'Q05',
       'Q06', 'Q07', 'Q08', 'Q09', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16',
       'Q17', 'Q18', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26',
       'Q27', 'Q28', 'Q69', 'Q70', 'Q71', 'Q72', 'Q73', 'Q74', 'DataCollected']]

In [284]:
f238.columns

Index(['Subject ID', 'SiteID', 'Visit', 'FormDate', 'Q01', 'Q02', 'Q03', 'Q04',
       'Q05', 'Q06', 'Q07', 'Q08', 'Q09', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14',
       'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24',
       'Q25', 'Q26', 'Q27', 'Q28', 'Q69', 'Q70', 'Q71', 'Q72', 'Q73', 'Q74',
       'DataCollected'],
      dtype='object')

In [285]:
f238.columns[4]

'Q01'

In [272]:
for i, k in enumerate(f238.columns):
    print(i, k)

0 Subject ID
1 SiteID
2 Visit
3 FormDate
4 Q01
5 Q02
6 Q03
7 Q04
8 Q05
9 Q06
10 Q07
11 Q08
12 Q09
13 Q10
14 Q11
15 Q12
16 Q13
17 Q14
18 Q15
19 Q16
20 Q17
21 Q18
22 Q19
23 Q20
24 Q21
25 Q22
26 Q23
27 Q24
28 Q25
29 Q26
30 Q27
31 Q28
32 Q69
33 Q70
34 Q71
35 Q72
36 Q73
37 Q74
38 DataCollected


In [ ]:
f238.head(5)

In [143]:
filename = r'//FS2.smpp.local\RTO\CIS-PD MUSC\datadictionary.sas7bdat'
datadict = pd.read_sas(filename, format = 'sas7bdat', encoding='iso-8859-1')
datadict.columns = datadict.columns.str.replace('z', '')

In [279]:
# get question names
questions = datadict.loc[datadict.FormID == 238.0]
questions = questions[['FieldNb','FieldNm']]
questions = questions.reset_index(drop=True)

In [280]:
questions = questions.drop(questions.index[30:70]).reset_index(drop=True)
# remove z in FieldNb column
questions.FieldNb = questions.FieldNb.str.replace('z', '')
# change first 2 question names
questions.FieldNm[0] = questions.FieldNb[0]
questions.FieldNm[1] = questions.FieldNb[1]

In [251]:
def keepleftstring(string, sep=' '):
    """Take a string and keep text before specified character.
    Default character is ' '.
    """
    new_string = string.split(sep, 1)[0]
    return new_string

In [281]:
# change question descriptions
# skip indices: 0, 1, 2, 9, 36
for i, k in enumerate(questions.FieldNm):
    if not (i==0) | (i==1) | (i==2)| (i==9)| (i==36):
        questions.FieldNm[i]=keepleftstring(k)

In [282]:
# remove FieldNb rows: DataCollected, FormDate, and Notes
questions = questions.drop(questions.index[0:2]).reset_index(drop=True)
questions = questions.drop(questions.index[-1]).reset_index(drop=True)

# Rename f238 column names then save

In [300]:
f238.columns

Index(['Subject ID', 'SiteID', 'Visit', 'FormDate', 'Q01', 'Q02', 'Q03', 'Q04',
       'Q05', 'Q06', 'Q07', 'Q08', 'Q09', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14',
       'Q15', 'Q16', 'Q17', 'Q18', 'Q19', 'Q20', 'Q21', 'Q22', 'Q23', 'Q24',
       'Q25', 'Q26', 'Q27', 'Q28', 'Q69', 'Q70', 'Q71', 'Q72', 'Q73', 'Q74',
       'DataCollected'],
      dtype='object')

In [303]:
newcolnames = ['Subject ID', 'SiteID', 'Visit', 'FormDate',
               'Primary source of information', '1.1', '1.2', '1.3', '1.4', '1.5',
               '1.6', 'Who is filling out this questionnaire', '1.7', '1.8',
               '1.9', '1.10', '1.11', '1.12', '1.13', '2.1', '2.2', '2.3', '2.4',
               '2.5', '2.6', '2.7', '2.8', '2.9', '2.10', '2.11', '2.12', '2.13',
               '4.1', '4.2', '4.3', '4.4', '4.5', '4.6', 'DataCollected']
f238.columns = newcolnames

In [305]:
f238.columns = newcolnames

In [306]:
f238.columns

Index(['Subject ID', 'SiteID', 'Visit', 'FormDate',
       'Primary source of information', '1.1', '1.2', '1.3', '1.4', '1.5',
       '1.6', 'Who is filling out this questionnaire', '1.7', '1.8', '1.9',
       '1.10', '1.11', '1.12', '1.13', '2.1', '2.2', '2.3', '2.4', '2.5',
       '2.6', '2.7', '2.8', '2.9', '2.10', '2.11', '2.12', '2.13', '4.1',
       '4.2', '4.3', '4.4', '4.5', '4.6', 'DataCollected'],
      dtype='object')

In [297]:
questions.FieldNm.values

array(['Primary source of information', '1.1', '1.2', '1.3', '1.4', '1.5',
       '1.6', 'Who is filling out this questionnaire', '1.7', '1.8',
       '1.9', '1.10', '1.11', '1.12', '1.13', '2.1', '2.2', '2.3', '2.4',
       '2.5', '2.6', '2.7', '2.8', '2.9', '2.10', '2.11', '2.12', '2.13',
       '4.1', '4.2', '4.3', '4.4', '4.5', '4.6'], dtype=object)

In [ ]:
f238

In [308]:
# save file as updrs_124.h5
filename = r'//FS2.smpp.local\RTO\CIS-PD MUSC\decoded_forms\updrs_124.h5'
# save pd.read_hdf(filename)
with open(filename,'wb') as f:
    f238.to_hdf(filename, key='f238', mode='w')

C:\ProgramData\anaconda3\lib\site-packages\pandas\core\generic.py:1993: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['Visit', 'Primary source of information', 'Who is filling out this questionnaire', 'DataCollected']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [ ]:
filename = r'//FS2.smpp.local\RTO\CIS-PD MUSC\decoded_forms\updrs_124.h5'
pd.read_hdf(filename, 'f238')

# Explore datadict for form 238

In [254]:
temp = datadict.loc[datadict.FormID == 238.0]

In [ ]:
temp.loc[50:60]